In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Translation Evaluation with reference

## Install Vertex AI SDK for Python

In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform[evaluation]

## Authenticate to access GCP

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id="ai-hangsik")

    from google.colab import drive
    drive.mount('/content/drive')

!gcloud config set project ai-hangsik

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Updated property [core/project].


## Initialize Vertex AI and Gemini model

In [3]:
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
#MODEL = "gemini-pro-experimental"
MODEL = "gemini-1.5-pro"

import base64
import vertexai
from vertexai.generative_models import GenerativeModel

import pandas as pd
pd.options.display.max_colwidth = 300


vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel(MODEL)

## Pointwise evaluation

### Test dataset

In [4]:
# Pointwise 검사를 위한 데이터셋 준비
# Pointwise는 단일 모델의 결과에 대한 평가입니다.
data = [
    ["This account is not protected with a two-factor authentication.",
     "2차 인증이 되지 않은 계정입니다.",
     "이중 인증으로 보호되지 않은 계정입니다."],
    ["Even the longest journey begins with a single step.",
    "천 리 길도 한 걸음부터.",
    "가장 먼 여정도 한 걸음에서 시작됩니다."],
    ["Even the longest journey begins with a single step.",
    "Лиха беда начало.",
    "Даже самая длинная дорога продолжается с одного шага."]
]
pointwise_df = pd.DataFrame(data, columns=['prompt', 'reference', 'response'])
pointwise_df

,prompt,reference,response
0,This account is not protected with a two-factor authentication.,2차 인증이 되지 않은 계정입니다.,이중 인증으로 보호되지 않은 계정입니다.
1,Even the longest journey begins with a single step.,천 리 길도 한 걸음부터.,가장 먼 여정도 한 걸음에서 시작됩니다.
2,Even the longest journey begins with a single step.,Лиха беда начало.,Даже самая длинная дорога продолжается с одного шага.


In [6]:
from vertexai.evaluation import EvalTask, PointwiseMetric, PairwiseMetric
import vertexai.evaluation.metrics.pointwise_metric as pointwise_metric

pointwise_ground_truth_metric_prompt = """

# Instruction
당신은 전문적인 번역 품질 평가자 입니다. 당신은 AI 모델이 번역한 내용의 품질을 평가해야 합니다.
우리는 당신에게 'Source, AI translation, Ground Truth'를 제공할 것입니다.
업무를 수행하기 위해 'Source, AI translation, Ground Truth'을 신중하게 읽고, 아래의 'Evaluation' 섹션에 정의된 'Evaluation criteria'에 근거하여 평가해야 합니다.
당신은 'Rating rubric'과 'Evaluation steps'에 기반하여 평가해야 합니다. 평가의 이유에 대해 단계별로 '한글'로 설명하고 'Rating rubric'의 순위만 선택해야 합니다.

# Evaluation
## Metric Definition
번역이 'Ground Truth'와 동일하게 번역되었는지, 그렇지 않으면 의미적으로 유사하고 간결, 명확, 캐주얼하게 작성되었는지 전반적으로 확인합니다.

## Evaluation criteria
예시 일치성: 예시로 제공되는 'Ground Truth'와 얼마나 일치하는지 확인합니다.
지시를 잘 따르는지: 'AI translation'이 'Evaluation Definition'을 잘 따라서 수행됐는지 확인합니다.

## Rating rubric
5: (매우 좋음). 'Ground Truth'와 글자수 등 표현이 정확하게 일치함
4: (좋음). 'Ground Truth'와 어순 외 차이 없음
3: (괜찮음). 속담이나 관용적 표현을 사용하지 못했지만 이해하는데 큰 문제 없음
2: (나쁨). 이해하기가 어려움
1: (매우 나쁨). 번역결과가 'Ground Truth'와 매우 다름

## Evaluation steps
STEP 1: 'Ground Truth'을 참고하여 'AI translation'이 번역한 내용을 비교합니다.
STEP 2: 평가 규칙에 따라 점수를 부여합니다.

# Source, AI translation, Ground Truth
### Source
{prompt}

### Ground Truth
{reference}

### AI translation
{response}
"""

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.evaluation.PointwiseMetric

pointwise_ground_truth_text_quality = PointwiseMetric(
    metric="pointwise_ground_truth_quality",
    metric_prompt_template=pointwise_ground_truth_metric_prompt,
)
#For MetricX, require source field
pointwise_df['source'] = pointwise_df['prompt']

eval_task = EvalTask(dataset=pointwise_df,
                    metrics=[pointwise_ground_truth_text_quality, "bleu", "rouge", pointwise_metric.MetricX(), pointwise_metric.Comet()],
                    experiment="pointwise-eval")
result_pointwise = eval_task.evaluate()

#comet, higher is better (0-1)
#metricx, lower is better (0-25)
result_pointwise.metrics_table

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/pointwise-eval-f34785a1-543d-4c34-9d84-60efa18f13d0 to Experiment: pointwise-eval


INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 15 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 15/15 [01:08<00:00,  4.55s/it]
INFO:vertexai.evaluation._evaluation:Evaluation Took:68.22712948900153 seconds


,prompt,reference,response,source,pointwise_ground_truth_quality/explanation,pointwise_ground_truth_quality/score,bleu/score,rouge/score,metricx/score,comet/score
0,This account is not protected with a two-factor authentication.,2차 인증이 되지 않은 계정입니다.,이중 인증으로 보호되지 않은 계정입니다.,This account is not protected with a two-factor authentication.,"STEP 1: 'Ground Truth'와 'AI translation'을 비교했습니다.\n'Ground Truth'는 ""2차 인증이 되지 않은 계정입니다.""이고, 'AI translation'은 ""이중 인증으로 보호되지 않은 계정입니다.""입니다.\nSTEP 2: 평가 규칙에 따라 점수를 부여합니다.\n'AI translation'이 'Ground Truth'와 의미적으로 유사하고 간결, 명확, 캐주얼하게 작성되었으므로 4점으로 평가합니다.",4.0,0.302138,0.0,0.961893,0.915181
1,Even the longest journey begins with a single step.,천 리 길도 한 걸음부터.,가장 먼 여정도 한 걸음에서 시작됩니다.,Even the longest journey begins with a single step.,"STEP 1: 'Ground Truth'는 속담을 사용하여 표현했지만, 'AI translation'은 있는 그대로 직역했습니다.\nSTEP 2: 'AI translation'은 'Ground Truth'와 의미가 다르기 때문에 ""나쁨""으로 평가합니다.",2.0,0.078099,0.0,NaN,0.925086
2,Even the longest journey begins with a single step.,Лиха беда начало.,Даже самая длинная дорога продолжается с одного шага.,Even the longest journey begins with a single step.,"STEP 1: AI는 ""가장 긴 여정도 한 걸음부터 시작된다""라는 속담을 ""가장 긴 길도 한 걸음부터 계속된다""로 어색하게 번역했습니다. Ground Truth는 러시아 속담인 ""모든 시작이 어렵다""라는 의미로, AI는 원문의 의미를 제대로 반영하지 못했습니다.\nSTEP 2: AI 번역은 Ground Truth와 의미가 전혀 다르므로 1점으로 평가합니다.",1.0,0.047677,0.0,NaN,0.568028


## Pairwise evaluation

In [7]:
# Pairwise 검사를 위한 데이터셋 준비
# Pairwise 는 2개 모델의 성능을 비교하기 위한 방법입니다.
#prompt(Source), reference, response (Response B), baseline_model_response (Response A)
data = [
    ["This account is not protected with a two-factor authentication.",
     "2차 인증이 되지 않은 계정입니다.",
     "이중 인증으로 보호되지 않은 계정입니다.",
     "이 계정은 이중 인증으로 안전하게 보호되지 않습니다."],
    ["Even the longest journey begins with a single step.",
    "천 리 길도 한 걸음부터.",
    "가장 먼 여정도 한 걸음에서 시작됩니다.",
    "가장 긴 여행조차도 한 걸음부터 시작됩니다."],
    ["Even the longest journey begins with a single step.",
    "Лиха беда начало.",
    "Даже самая длинная дорога продолжается с одного шага.",
    "Даже самый долгий путь начинается с первого шага."]
]
pairwise_df = pd.DataFrame(data, columns=['prompt', 'reference', 'response', 'baseline_model_response'])
pairwise_df

,prompt,reference,response,baseline_model_response
0,This account is not protected with a two-factor authentication.,2차 인증이 되지 않은 계정입니다.,이중 인증으로 보호되지 않은 계정입니다.,이 계정은 이중 인증으로 안전하게 보호되지 않습니다.
1,Even the longest journey begins with a single step.,천 리 길도 한 걸음부터.,가장 먼 여정도 한 걸음에서 시작됩니다.,가장 긴 여행조차도 한 걸음부터 시작됩니다.
2,Even the longest journey begins with a single step.,Лиха беда начало.,Даже самая длинная дорога продолжается с одного шага.,Даже самый долгий путь начинается с первого шага.


In [8]:
pairwise_model_compare_metric_prompt = """
# Instruction
당신은 전문적인 번역 품질 평가자 입니다. 당신은 2개의 AI 모델이 번역한 내용의 품질을 평가해야 합니다.
우리는 당신에게 'Source, Response A, Response B, Ground Truth'를 제공할 것입니다.
업무를 수행하기 위해 'Source, Response A, Response B, Ground Truth'을 신중하게 읽고, 아래의 '평가' 섹션에 정의된 '평가항목'에 근거하여 평가해야 합니다.
당신은 'Evaluation rule'과 'Evaluation steps'에 기반하여 평가해야 합니다. 평가의 이유에 대해 단계별로 '한글'로 설명하고 'Evaluation rule'의 순위만 선택해야 합니다.

# Evaluation
## Metric Definition
번역이 'Ground Truth'와 동일하게 번역되었는지, 그렇지 않으면 의미적으로 유사하고 간결, 명확, 캐주얼하게 작성되었는지 전반적으로 확인합니다.

## Evaluation criteria
예시 일치성: 예시로 제공되는 'Ground Truth'와 얼마나 일치하는지 확인합니다.
지시를 잘 따르는지: 'AI translation'이 'Evaluation Definition'을 잘 따라서 수행됐는지 확인합니다.

## Rating rubric
STEP 1: Analyze Response A based on all the Criteria.
STEP 2: Analyze Response B based on all the Criteria.
STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
STEP 5: Output your assessment reasoning in the explanation field.

# Source, Response A, Response B, Ground Truth
### Source
{prompt}

### Ground Truth
{reference}

### Response A
{baseline_model_response}

### Response B
{response}
"""

pairwise_ground_truth_text_quality = PairwiseMetric(
    metric="pairwise_ground_truth_quality",
    metric_prompt_template=pairwise_model_compare_metric_prompt,
)

eval_task = EvalTask(dataset=pairwise_df,
                    metrics=[pairwise_ground_truth_text_quality],
                    experiment="pairwise-eval")
result_pairwise = eval_task.evaluate()
result_pairwise.metrics_table.rename(columns={"baseline_model_response": "Response A", "response": "Response B"}).replace(['BASELINE', 'CANDIDATE'], ['A', 'B'])

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/pairwise-eval-a328d82f-b501-4410-a179-ff71ebdb1b5a to Experiment: pairwise-eval


INFO:vertexai.evaluation._evaluation:Computing metrics with a total of 3 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 3/3 [00:07<00:00,  2.46s/it]
INFO:vertexai.evaluation._evaluation:All 3 metric requests are successfully computed.
INFO:vertexai.evaluation._evaluation:Evaluation Took:7.402775962000305 seconds


,prompt,reference,Response B,Response A,pairwise_ground_truth_quality/explanation,pairwise_ground_truth_quality/pairwise_choice
0,This account is not protected with a two-factor authentication.,2차 인증이 되지 않은 계정입니다.,이중 인증으로 보호되지 않은 계정입니다.,이 계정은 이중 인증으로 안전하게 보호되지 않습니다.,"Response A는 '이중 인증으로 안전하게 보호되지 않습니다.'라고 번역했는데, Ground Truth와 비교했을 때 '안전하게'라는 단어가 추가되어 불필요하게 길고, '보호되지 않습니다'라는 수동형 표현이 어색합니다. Response B는 '이중 인증으로 보호되지 않은 계정입니다.'라고 번역했는데, Ground Truth와 의미가 같고 간결하고 명확하며 자연스럽습니다. 따라서 Response B가 Ground Truth와 더 유사하고 자연스러운 번역입니다.",B
1,Even the longest journey begins with a single step.,천 리 길도 한 걸음부터.,가장 먼 여정도 한 걸음에서 시작됩니다.,가장 긴 여행조차도 한 걸음부터 시작됩니다.,"Response A와 B 모두 자연스러운 번역이지만, Response B가 속담의 의미를 더 잘 살리고 있기 때문입니다. '먼 여정'이라는 표현이 '긴 여정'보다 속담의 함축적 의미인 '어려운 일'이나 '큰 목표'에 더 가깝게 느껴집니다. Response A는 '가장 긴 여행'이라는 표현이 다소 평범하게 들립니다.",B
2,Even the longest journey begins with a single step.,Лиха беда начало.,Даже самая длинная дорога продолжается с одного шага.,Даже самый долгий путь начинается с первого шага.,"Response A는 '가장 긴 여정도 한 걸음부터 시작된다'라는 원문의 의미를 잘 살려 러시아어로 번역했지만, Response B는 '가장 긴 길도 한 걸음으로 계속된다'라는 어색한 의미로 번역되었습니다. Ground Truth인 'Лиха беда начало'는 '시작이 어려울 뿐이다' 또는 '시작이 반이다'라는 러시아 속담으로, Response A가 원문의 의미와 Ground Truth의 속담 의미 모두에 더 가깝습니다.",A
